**-- Giới Thiệu --**

Như đã giải thích trong Section 2.4, vi phân là phép tính thiết yếu trong hầu như tất cả mọi thuật toán học sâu. Mặc dù các phép toán trong việc tính đạo hàm khá trực quan và chỉ yêu cầu một chút kiến thức giải tích, nhưng với các mô hình phức tạp, việc tự tính rõ ràng từng bước khá là mệt (và thường rất dễ sai).

Gói thư viện autograd giải quyết vấn đề này một cách nhanh chóng và hiệu quả bằng cách tự động hoá các phép tính đạo hàm (automatic differentiation). Trong khi nhiều thư viện yêu cầu ta phải biên dịch một đồ thị biểu tượng (symbolic graph) để có thể tự động tính đạo hàm, autograd cho phép ta tính đạo hàm ngay lập tức thông qua các dòng lệnh thông thường. Mỗi khi đưa dữ liệu chạy qua mô hình, autograd xây dựng một đồ thị và theo dõi xem dữ liệu nào kết hợp với các phép tính nào để tạo ra kết quả. Với đồ thị này autograd sau đó có thể lan truyền ngược gradient lại theo ý muốn. Lan truyền ngược ở đây chỉ đơn thuần là truy ngược lại đồ thị tính toán và điền vào đó các giá trị đạo hàm riêng theo từng tham số.

In [71]:
import torch

# 2.5.1. Một ví dụ đơn giản

Lấy ví dụ đơn giản, giả sử chúng ta muốn tính vi phân của hàm số $y=2x^⊤x$ theo vector cột $x$. Để bắt đầu, ta sẽ tạo biến x và gán cho nó một giá trị ban đầu.

In [72]:
x = torch.arange(4, dtype = torch.float32)
x

tensor([0., 1., 2., 3.])

Lưu ý rằng trước khi có thể tính gradient của $y$ theo $x$ , chúng ta cần một nơi để lưu giữ nó. Điều quan trọng là ta không được cấp phát thêm bộ nhớ mới mỗi khi tính đạo hàm theo một biến xác định, vì ta thường cập nhật cùng một tham số hàng ngàn vạn lần và sẽ nhanh chóng dùng hết bộ nhớ.

Cũng lưu ý rằng, bản thân giá trị gradient của hàm số đơn trị theo một vector  x  cũng là một vector với cùng kích thước. Do vậy trong mã nguồn sẽ trực quan hơn nếu chúng ta lưu giá trị gradient tính theo x dưới dạng một thuộc tính của chính ndarray x. Chúng ta cấp bộ nhớ cho gradient của một ndarray bằng cách gọi phương thức attach_grad.

In [73]:
# Can also create x = torch.arange(4.0, requires_grad=True)
x.requires_grad_(True)
x.grad  # The gradient is None by default

Giờ ta sẽ tính $y$ theo $x$

In [74]:
y = 2 * torch.dot(x,x)
y

tensor(28., grad_fn=<MulBackward0>)

Bởi vì $x$ là một tensor có độ dài bằng 4, torch.dot sẽ tính toán tích vô hướng của $x$ và $x$, trả về một số vô hướng mà sẽ được gán cho y. Tiếp theo, ta có thể tính toán gradient của y theo mỗi thành phần của $x$ một cách tự động bằng cách gọi hàm `backward` của y.

In [75]:
y.backward()

Nếu kiểm tra lại giá trị của x.grad, ta sẽ thấy nó đã được ghi đè bằng gradient mới được tính toán.

In [76]:
x.grad

tensor([ 0.,  4.,  8., 12.])

Gradient của hàm $y=2x^⊤x$ theo $x$ phải là $4x$. Hãy kiểm tra một cách nhanh chóng rằng giá trị gradient mong muốn được tính toán đúng. Nếu hai tensor là giống nhau, thì mọi cặp phần tử tương ứng cũng bằng nhau.

In [77]:
x.grad == 4*x

tensor([True, True, True, True])

Nếu ta tiếp tục tính gradient của một biến khác mà giá trị của nó là kết quả của một hàm theo biến x, thì nội dung trong x.grad sẽ bị ghi đè. Ngoài ra ta cũng có thể reset bộ nhớ của nó bằng hàm `x.grad.zero()`

In [78]:
""" Thắc mắc """
# x.grad.zero_()
print(x.grad)
y = x.sum()
y.backward()
print(x.grad, y)

tensor([ 0.,  4.,  8., 12.])
tensor([ 1.,  5.,  9., 13.]) tensor(6., grad_fn=<SumBackward0>)


# 2.5.2. Truyền ngược cho các biến không phải Số vô hướng

Về mặt kỹ thuật, khi y không phải một số vô hướng, cách diễn giải tự nhiên nhất cho vi phân của một vector y theo vector x đó là một ma trận. Với các bậc và chiều cao hơn của y và x, kết quả của phép vi phân có thể là một tensor bậc cao.

Tuy nhiên, trong khi những đối tượng như trên xuất hiện trong học máy nâng cao (bao gồm học sâu), thường thì khi ta gọi lan truyền ngược trên một vector, ta đang cố tính toán đạo hàm của hàm mất mát theo mỗi batch bao gồm một vài mẫu huấn luyện. Ở đây, ý định của ta không phải là tính toán ma trận vi phân mà là tổng của các đạo hàm riêng được tính toán một cách độc lập cho mỗi mẫu trong batch.

Vậy nên khi ta gọi backward lên một biến vector y – là một hàm của x, MXNet sẽ cho rằng ta muốn tính tổng của các gradient. Nói ngắn gọn, MXNet sẽ tạo một biến mới có giá trị là số vô hướng bằng cách cộng lại các phần tử trong y và tính gradient theo x của biến mới này.

In [79]:
# x.grad.zero_()
# y = x * x
# print(y)
# y.backward(gradient=torch.ones(len(y)))  # Faster: y.sum().backward()
# x.grad

x.grad.zero_()
y = torch.dot(x,x)
y.backward()  # Faster: y.sum().backward()
x.grad == 2*x

tensor([True, True, True, True])

# 2.5.3. Tách rời Tính toán

Đôi khi chúng ta muốn chuyển một số phép tính ra khỏi đồ thị tính toán. Ví dụ, giả sử y đã được tính như một hàm của x, rồi sau đó z được tính như một hàm của cả y và x. Bây giờ, giả sử ta muốn tính gradient của z theo x, nhưng vì lý do nào đó ta lại muốn xem y như là một hằng số và chỉ xét đến vai trò của x như là biến số của z sau khi giá trị của y đã được tính.

Trong trường hợp này, ta có thể gọi u = y.detach() để trả về một biến u mới có cùng giá trị như y nhưng không còn chứa các thông tin về cách mà y đã được tính trong đồ thị tính toán. Nói cách khác, gradient sẽ không thể chảy ngược qua u về x được. Bằng cách này, ta đã tính u như một hàm của x ở ngoài phạm vi của autograd.record, dẫn đến việc biến u sẽ được xem như là một hằng số mỗi khi ta gọi backward. Chính vì vậy, hàm backward sau đây sẽ tính đạo hàm riêng của z = u * x theo x khi xem u như là một hằng số, thay vì đạo hàm riêng của z = x * x * x theo x.

In [80]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u*x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

Bởi vì sự tính toán của y đã được ghi lại, chúng ta có thể gọi y.backward() sau đó để lấy đạo hàm của y = x * x theo x, tức là 2 * x.

In [81]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

# 2.5.4. Tính gradient của Luồng điều khiển Python

Một lợi thế của việc sử dụng vi phân tự động là khi việc xây dựng đồ thị tính toán đòi hỏi trải qua một loạt các câu lệnh điều khiển luồng Python, (ví dụ như câu lệnh điều kiện, vòng lặp và các lệnh gọi hàm tùy ý), ta vẫn có thể tính gradient của biến kết quả. Trong đoạn mã sau, hãy lưu ý rằng số lần lặp của vòng lặp while và kết quả của câu lệnh if đều phụ thuộc vào giá trị của đầu vào a.

In [84]:
def f(a):
  b = a * 2
  while b.norm() < 1000:
    b = b * 2
  if b.sum() > 0:
    c = b 
  else:
    c = 100*b 
  return c


Trước khi chạy, ta tạo 1 tập input ngẫu nhiên nên do đó không biết được dạng đồ thị sẽ nhận được. 

In [85]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

Giờ ta có thể phân tích hàm f được định nghĩa ở phía trên. Hãy để ý rằng hàm này tuyến tính từng khúc theo đầu vào a. Nói cách khác, với mọi giá trị của a tồn tại một hằng số k sao cho $f(a) = k * a$, ở đó giá trị của k phụ thuộc vào đầu vào a. Do đó, ta có thể kiểm tra giá trị của gradient bằng cách tính $d / a$.

In [86]:
a.grad == d/a

tensor(True)